# Transition-based arc-eager unlabeled dependency parser for Ukrainian

## Read the data

Useful links:
* [UD corpus for Ukrainian](https://github.com/UniversalDependencies/UD_Ukrainian-IU/)
* [Easy-to-use library for parsing UD](https://github.com/EmilStenstrom/conllu)

In [1]:
from collections import OrderedDict
from conllu import parse
from enum import Enum

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

In [2]:
PATH = "./UD_Ukrainian-IU"

with open(PATH + "/uk_iu-ud-train.conllu", "r") as f:
    trees = parse(f.read())

with open(PATH + "/uk_iu-ud-test.conllu", "r") as f:
    test_trees = parse(f.read())

ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

print(len(trees), 'train sentences')
print(len(test_trees), 'test sentences')

5496 train sentences
892 test sentences


In [3]:
tree = trees[0]
print(tree)

tree = trees[0]
for node in tree:
    head = node["head"]
    print("{} {} <-- {} -- {}".format(node["id"],
                                     node["form"],
                                     node["deprel"],
                                     tree[head - 1]["form"]
                                     if head > 0 else "root"))

TokenList<У, домі, римського, патриція, Руфіна, була, прегарна, фреска, ,, зображення, Венери, та, Адоніса, .>
1 У <-- case -- домі
2 домі <-- obl -- була
3 римського <-- amod -- патриція
4 патриція <-- nmod -- домі
5 Руфіна <-- flat:title -- патриція
6 була <-- root -- root
7 прегарна <-- amod -- фреска
8 фреска <-- nsubj -- була
9 , <-- punct -- зображення
10 зображення <-- appos -- фреска
11 Венери <-- nmod -- зображення
12 та <-- cc -- Адоніса
13 Адоніса <-- conj -- Венери
14 . <-- punct -- була


## Design actions and the oracle

In [4]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"

In [5]:
def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [6]:
def trace_actions(tree, log=True):
    """
    Try out the oracle to verify it's returning the right actions.
    """
    stack, queue, relations, rel_types = [ROOT], tree[:], [], []
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        if log:
            print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
            print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
            print("Relations:", relations)
            print("Rel.Types:", rel_types)
            print(action)
            print("========================")
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            rel_types.append(stack[-1]["deprel"])
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            rel_types.append(queue[0]["deprel"])
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    if log:
        print("Gold relations:")
        print([(node["id"], node["head"]) for node in tree])
        print("Retrieved relations:")
        print(sorted(relations))

trace_actions(tree)

Stack: ['ROOT_0']
Queue: ['У_1', 'домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Rel.Types: []
Actions.SHIFT
Stack: ['ROOT_0', 'У_1']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Rel.Types: []
Actions.LEFT
Stack: ['ROOT_0']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Rel.Types: ['case']
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2']
Queue: ['римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Rel.Types: ['case']
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2', 'римського_3']
Queue: ['патриція_4', 'Руфін

## Feature extraction

Reference: [Dependency Parsing by Kübler, McDonald, and Nivre](https://books.google.com.ua/books?id=k3iiup7HB9UC&pg=PA21&hl=uk&source=gbs_toc_r&cad=4#v=onepage&q&f=false)

In [7]:
def extract_features_baseline(stack, queue):
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
        if stack_top["feats"]:
            for k, v in stack_top["feats"].items():
                features["s0-" + k] = v
    if len(stack) > 1:
        features["s1-word"] = stack[-2]["form"]
        features["s1-tag"] = stack[-2]["upostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        if queue_top["feats"]:
            for k, v in queue_top["feats"].items():
                features["q0-" + k] = v
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    if stack and queue:
        features["distance"] = queue[0]["id"] - stack[-1]["id"]
    return features

> I extended feaure extraction (see below):

In [9]:
def extract_features(stack, queue):
    features = dict()
    for i in range(0, min(len(stack),3)):
        el = stack[-i-1]
        features[f"s{i}-word"] = el["form"]
        features[f"s{i}-lemma"] = el["lemma"]
        features[f"s{i}-tag"] = el["upostag"]
        features[f"s{i}-wordtag"] = el["form"] + el["upostag"]
        if el["feats"]:
            for k, v in el["feats"].items():
                features[f"s{i}-" + k] = v
    for i in range(1, len(stack)):
        features[f"s0-word_s{i}-word"] = stack[-1]["form"] + stack[-i-1]["form"]
        features[f"s0-tag_s{i}-tag"] = stack[-1]["upostag"] + stack[-i-1]["upostag"]
        features[f"s0-wordtag_s{i}-tag"] = stack[-1]["form"] + stack[-1]["upostag"] + stack[-i-1]["upostag"]
        features[f"s0-tag_s{i}-wordtag"] = stack[-1]["upostag"] + stack[-i-1]["form"] + stack[-i-1]["upostag"]
        features[f"s0s{i}-distance"] = stack[-1]["id"] - stack[-2]["id"]

    if queue:
        features["q0-word"] = queue[0]["form"]
        features["q0-lemma"] = queue[0]["lemma"]
        features["q0-tag"] = queue[0]["upostag"]
        if queue[0]["feats"]:
            for k, v in queue[0]["feats"].items():
                features["q0-" + k] = v
    if len(queue) > 1:
        features["q1-word"] = queue[1]["form"]
        features["q1-tag"] = queue[1]["upostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if stack and queue:
        features["s0-tag_q0-word"] = stack[-1]["upostag"] + queue[0]["form"]
        features["s0-wordtag_q0-tag"] = stack[-1]["form"] + stack[-1]["upostag"] + queue[0]["upostag"]
        features["s0q0-distance"] = queue[0]["id"] - stack[-1]["id"]
#     print('\nStack:', [s["form"] for s in stack])
#     print('Queue:', [q["form"] for q in queue])
#     print('Features:', features)
    return features

## Prepare train and test data

In [10]:
def get_data(tree):
    features, deps, deps_types = [], [], []
    stack, queue, relations = [ROOT], tree[:], []

    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        features.append(extract_features(stack, queue))
        deps.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
            deps_types.append('None')
        elif action == Actions.REDUCE:
            stack.pop()
            deps_types.append('None')
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            deps_types.append(stack[-1]["deprel"])
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            deps_types.append(queue[0]["deprel"])
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
            deps_types.append('None')
    return features, deps

In [11]:
features, labels = get_data(tree)
print("Number of words:", len(tree))
print("Number of actions:", len(labels))
print("List of actions taken:", labels)
print("Features: {} x {}".format(len(features), len(features[0])))

Number of words: 14
Number of actions: 29
List of actions taken: ['shift', 'left', 'shift', 'shift', 'left', 'right', 'right', 'reduce', 'reduce', 'left', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'right', 'shift', 'left', 'right', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features: 29 x 14


In [12]:
train_features, train_labels = [], []
for tree in trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

190298 190298


In [13]:
test_features, test_labels = [], []
for tree in test_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    test_features += tree_features
    test_labels += tree_labels

print(len(test_features), len(test_labels))

35124 35124


In [16]:
def dep_parse(sentence, oracle, vectorizer, log=True):
    stack, queue, relations = [ROOT], sentence[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
            if log:
                print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
                print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
                print("Relations:", relations)
                print(action)
                print("========================")
            # actual parsing
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

## Vectorize

In [17]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)

print("\nTotal number of features: ", len(vec.get_feature_names()))

train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)


Total number of features:  967093


## Train a classifier

### 1. Logistic Regression

In [47]:
# Baseline Result (from original notebook) is kept below

              precision    recall  f1-score   support

        left       0.87      0.90      0.88      8658
      reduce       0.86      0.80      0.83      9350
       right       0.79      0.82      0.80      8291
       shift       0.88      0.89      0.89      8825

    accuracy                           0.85     35124
   macro avg       0.85      0.85      0.85     35124
weighted avg       0.85      0.85      0.85     35124

Total: 17116
Correctly defined: 12373
UAS: 0.72


In [21]:
lrc = LogisticRegression(random_state=42, solver="sag",
                         multi_class="multinomial", max_iter=1000, verbose=1, 
                         n_jobs=-1)
lrc.fit(train_features_vectorized, train_labels)
predicted = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


max_iter reached after 409 seconds


/home/lochman/.virtualenvs/ml-py3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  6.8min finished


              precision    recall  f1-score   support

        left       0.90      0.93      0.91      8658
      reduce       0.88      0.83      0.85      9350
       right       0.82      0.83      0.82      8291
       shift       0.89      0.90      0.90      8825

    accuracy                           0.87     35124
   macro avg       0.87      0.87      0.87     35124
weighted avg       0.87      0.87      0.87     35124



In [27]:
try:
    total, tp = 0, 0
    for tree in test_trees:
        tree = [t for t in tree if type(t["id"])==int]
        golden = [(node["id"], node["head"]) for node in tree]
        predicted = dep_parse(tree, lrc, vec, log=False)
        total += len(tree)
        tp += len(set(golden).intersection(set(predicted)))

    print("Total:", total)
    print("Correctly defined:", tp)
    print("UAS:", round(tp / total, 2)) # the percentage of words in an input that are assigned the correct head
except:
    print("Stack became empty too early")

Stack became empty too early


## SVM

In [ ]:
svm = SVC(kernel='poly', degree=2,
        coef0=0, gamma=0.2, C=0.5,
        verbose=True, probability=True)

svm.fit(train_features_vectorized, train_labels)
predicted = svm.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

[LibSVM]

In [ ]:
try:
    total, tp = 0, 0
    for tree in test_trees:
        tree = [t for t in tree if type(t["id"])==int]
        golden = [(node["id"], node["head"]) for node in tree]
        predicted = dep_parse(tree, svm, vec, log=True)
        total += len(tree)
        tp += len(set(golden).intersection(set(predicted)))

    print("Total:", total)
    print("Correctly defined:", tp)
    print("UAS:", round(tp / total, 2)) # the percentage of words in an input that are assigned the correct head
except:
    print("Stack became empty too early")

## MLP

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=16, verbose=True,
                    learning_rate_init=0.1, max_iter=1000)

mlp.fit(train_features_vectorized, train_labels)
predicted = mlp.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

In [ ]:
try:
    total, tp = 0, 0
    for tree in test_trees:
        tree = [t for t in tree if type(t["id"])==int]
        golden = [(node["id"], node["head"]) for node in tree]
        predicted = dep_parse(tree, mlp, vec, log=False)
        total += len(tree)
        tp += len(set(golden).intersection(set(predicted)))

    print("Total:", total)
    print("Correctly defined:", tp)
    print("UAS:", round(tp / total, 2)) # the percentage of words in an input that are assigned the correct head
except:
    print("Stack became empty too early")

In [23]:
mlp = MLPClassifier(hidden_layer_sizes=16, verbose=True,
                    learning_rate_init=0.1, max_iter=1000)

mlp.fit(train_features_vectorized, train_labels)
predicted = mlp.predict(test_features_vectorized)
print(classification_report(test_labels, predicted))

/home/lochman/.virtualenvs/ml-py3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


              precision    recall  f1-score   support

        left       0.89      0.90      0.89      8658
      reduce       0.93      0.63      0.75      9350
       right       0.66      0.90      0.76      8291
       shift       0.89      0.89      0.89      8825

    accuracy                           0.82     35124
   macro avg       0.84      0.83      0.82     35124
weighted avg       0.85      0.82      0.82     35124



In [25]:
try:
    total, tp = 0, 0
    for tree in test_trees:
        tree = [t for t in tree if type(t["id"])==int]
        golden = [(node["id"], node["head"]) for node in tree]
        predicted = dep_parse(tree, mlp, vec, log=False)
        total += len(tree)
        tp += len(set(golden).intersection(set(predicted)))

    print("Total:", total)
    print("Correctly defined:", tp)
    print("UAS:", round(tp / total, 2)) # the percentage of words in an input that are assigned the correct head
except:
    print("Stack became empty too early")

Total: 17116
Correctly defined: 11678
UAS: 0.68


> The best performed is the baseline :( Logistic Regression with UAS: 0.72